In [6]:
import pandas as pd
a = pd.read_csv('./place_reviews.csv', encoding='utf-8')
a

,place_name,review
0,이호테우해수욕장,말 필요 없음
1,이호테우해수욕장,이호테우 해변 비오는 날 방문했는데 운치있고 넘 좋네요!!!!
2,이호테우해수욕장,좋았어요 ㅎㅎ 말등대도보공
3,이호테우해수욕장,귀여운 말등대가 있는 해수욕장🪅아이들 놀기 좋은 해수풀장이 있어요~
4,이호테우해수욕장,제주도 마지막 여행지
...,...,...
4186,해거름마을공원,풍경 좋아요 잠깐 쉬어가기 좋은 카페도 있어요
4187,노형근린공원,공원 강아지랑 걷기도 좋고 넓고 숲도 많아서 자연구경 딱좋아요
4188,서복공원,조금 아쉽네요
4189,서복공원,정방폭포 봤다면 여기까지 둘러보기~ 정원 굿~~


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

# CSV 파일 불러오기
df = pd.read_csv('./place_reviews.csv', encoding='utf-8')

stopwords = set(["의", "가", "이", "은", "들", "는", "좀", "잘", "걍", "과", "도", "를", "으로", "자", "에", "와", "한", "하다", "진짜", "여기", "너무", "youtu", "https", "좋아요", "좋고", "좋습니다",
                 "없어요", "엄청", "많이", "정말", "ㅎㅎ " ,"매우", "좋음", "코로나", "없음", "갑니다", "ㅋㅋ", "ㅠㅠ",
                 "같아요", "많고", "매우", "좋았어요", "있어서", "좋은", "있는"])

# 토픽-키워드 매핑 사전
topic_keywords = {
    "분위기": ["분위기", "카페", "음악", "조명", "인테리어"],
    "로맨틱": ["캔들", "와인", "재즈", "수영장", "밤하늘"],
    "화려한": ["불빛", "무대", "파티", "음악", "공연"],
    "아늑한": ["벽난로", "깔끔한 인테리어", "쿠션", "차", "책"],
    "모험적인": ["극한 스포츠", "야생동물", "체험", "캠핑", "모험"],
    "신비로운": ["신비로운 풍경", "초승달", "천체 관측", "고대 유적지", "미스터리"],
    "편안한": ["해변 의자", "휴양지", "해먹", "해수욕", "마사지"],
    "현대적인": ["스카이라인", "건축", "미술관", "박물관", "고급 호텔"],
    "고요한": ["계곡", "폭포", "산", "호수", "산책"],
    "전통적인": ["역사적인 건물", "민속 마을", "문화 축제", "전통적인 요리", "전통적인 예술"],
    "활기찬": ["클럽", "음악 축제", "시장", "바", "스포츠 경기"],
    "이국적인":["이국적인 해변", "국립공원", "이국적인 도시", "적도 국가", "사막"],
    "로컬한": ["로컬 음식", "로컬 시장", "지역 축제", "지역 예술", "지역 건축"],
    "청량한": ["산악 지대", "해변", "하천", "산책로", "공원"],
    "친환경적인": ["생태 관광", "친환경 호텔", "자연 보호", "지속 가능한 여행", "친환경적인 운송수단"],
    "힐링": ["힐링", "휴식", "여행", "휴가", "산책"],
    "인스타감성": ["인스타", "사진", "셀카", "감성", "포토"],
    "자연친화적인": ["자연", "숲", "나무", "야생화", "생태계"],
    "감성적인": ["분위기", "예술", "영화", "작은 카페", "시"],
    "레트로": ["빈티지", "올드스쿨", "LP 바", "키치", "고전적인"],
    "다채로운": ["다양한 문화", "음식 체험", "예술", "색다른 경험", "다양한 사람들"],
    "세계적인": ["유네스코", "세계 유산", "세계적인 명소", "역사", "기념물"],
    "스포티한": ["스포츠", "트레킹", "라이딩", "요가", "헬스"],
    "명상적인": ["명상", "요가", "휴식", "조용한 공간", "자기 성찰"],
    "역사적인": ["역사", "유적", "문화재", "국립 박물관", "고고학"],
    "아트": ["갤러리", "예술", "뮤지엄", "아티스트", "공예"],
    "리조트": ["스파", "휴양", "풀장", "리조트 호텔", "레져"],
    "도시적인": ["도시 탐험", "번화가", "쇼핑", "맛집", "영화관"],
    "농촌적인": ["농촌", "전원 마을", "농장 체험", "가족 여행", "자연과 함께"],
    "학문적인": ["학문", "도서관", "강연", "학술 세미나", "연구 기관"],
    "아이와 함께": ["아이와", "놀이공원", "가족 여행", "어린이 박물관", "동물원"],
    "배움": ["체험", "장인", "워크샵", "전통 기술", "문화 교류"],
    "건강한": ["웰니스", "건강", "스파", "온천", "디톡스"]
}

# 토픽별 리뷰 카운트를 저장할 딕셔너리
topic_counts = defaultdict(int)

# 리뷰 데이터 추출 및 키워드 추출
reviews = df['review'].tolist()

# KeyBERT 모델 초기화
model = KeyBERT('distilbert-base-nli-mean-tokens')

# 리뷰 데이터에서 키워드 추출
keywords = model.extract_keywords(" ".join(reviews), keyphrase_ngram_range=(1, 1), stop_words=stopwords)

# 리뷰를 토픽별로 분류
for review in reviews:
    assigned_topic = False
    for topic, keywords in topic_keywords.items():
        if any(keyword in review for keyword in keywords):
            topic_counts[topic] += 1
            assigned_topic = True
            break
    if not assigned_topic:
        topic_counts["기타"] += 1
        
# 토픽별 리뷰 카운트 출력
for topic, count in topic_counts.items():
    print(f"{topic}: {count}개")

C:\Users\lhe33\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ㅎㅎ'] not in stop_words.
  warnings.warn(


기타: 1514개
청량한: 61개
편안한: 34개
힐링: 108개
아늑한: 594개
감성적인: 519개
활기찬: 325개
리조트: 9개
인스타감성: 156개
분위기: 320개
고요한: 151개
자연친화적인: 79개
도시적인: 27개
모험적인: 103개
로맨틱: 98개
현대적인: 39개
세계적인: 2개
아이와 함께: 5개
화려한: 31개
아트: 9개
스포티한: 2개
배움: 2개
건강한: 3개
